# Data segmentation

## Dependencies and helper functions

In [ ]:
def apply_segmentation(image):
    image_normalized = image/np.max(image)
    seg = morphological_chan_vese(image_normalized , num_iter= 200, init_level_set='disk', smoothing=2, lambda1 = 10., lambda2 = 1.)
    return seg

## Loading of the data

In [ ]:
# Segment all images in the dataset on channel 9 (# ! Takes ~1h to run) (B03 - B08) / (B03 + B08)
l_segmented_images = paral(apply_segmentation, array_data_cropped[:,:,:,9], 10) #array_data_cropped[:,:,:,9] #np.squeeze(ndwi_array)+1
array_segmented_images = np.array(l_segmented_images)
del l_segmented_images

# Store result as it's pretty heavy to compute
with open(r"C:\Users\Carlos\pro\wekeo\Data engineering\\segmentation.npy", 'wb') as f:
    np.save(f, array_segmented_images)

## Filter out bad-segments based on size

In [ ]:
# Filter out segments that are 80% smaller than the main segment
for index, segment in enumerate(array_segmented_images):
    try:
        cnts = cv2.findContours(segment.astype('uint8'), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
        rect_areas = []
        for c in cnts:
            (x, y, w, h) = cv2.boundingRect(c)
            rect_areas.append(w * h)
        max_area = np.max(rect_areas)
        for c in cnts:
            (x, y, w, h) = cv2.boundingRect(c)
            cnt_area = w * h
            if cnt_area < 0.2 * max_area:
                segment[y:y + h, x:x + w] = 0
        array_segmented_images[index] = segment
    except Exception as e:
        print(e)

In [ ]:
# Filter out segments out segments that are outside of the largest segment when the lake is the 
# fullest (among the first images, as it becomes empty afterwards)
biggest_segment_index = np.argmax([np.sum(x) for x in array_segmented_images[:20]])
biggest_segment = array_segmented_images[biggest_segment_index]
for index, segment in enumerate(array_segmented_images):
    segment_diff = biggest_segment - segment
    segment[segment_diff < 0] = 0
    array_segmented_images[index] = segment

## Filtering out based on water content

In [ ]:
# Display distribution of intensity difference with 'pure' lake
fig, ax = plt.subplots(1, figsize = (10,5))
l_diff_mean_segments = [np.mean(array_data_cropped[index,:,:,9][segment==1])-np.mean(array_data_cropped[biggest_segment_index,:,:,9][biggest_segment==1]) if np.sum(segment)>0 else np.nan for index, segment in enumerate(array_segmented_images)]
fig.patch.set_facecolor('white')
plt.hist(l_diff_mean_segments, bins=100)
plt.ylim(0, 8)
plt.xlim(-100, 1600)
plt.xlabel("Segment mean intensity value difference w.r.t clean segment")
plt.ylabel("Frequency")
tresh = 1450

# Create one rectangle patch and add it to the plot
rect = patches.Rectangle((tresh, 0), 1600-tresh, 8, alpha = 0.3, facecolor="red")
ax.add_patch(rect)
plt.title("Distribution of segment differences of intensity value w.r.t cleanest segment")
plt.show()

In [ ]:
# Filter out segments that have an intensity which is significantly different from when the lake is the purest
array_segmented_images = np.array([segment if (l_diff_mean_segments[index]< tresh and not np.isnan(l_diff_mean_segments[index])) else np.zeros_like(segment) for index, segment in enumerate(array_segmented_images)])
